In [1]:
import numpy as np
import numpy.ma as ma
import os
import h5py
from skimage import io
import matplotlib
import matplotlib.pyplot as plt 
import pandas as pd
import math
import time
import sys
import pickle
from visdom import Visdom

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
from torchsummary import summary
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

from collections import defaultdict
from sklearn.metrics import f1_score
import cv2
import pandas as pd
import h5py

############### Visualization

from ipywidgets import Button, Layout
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display,clear_output,Video
from copy import copy
import numpy.ma as ma
import matplotlib.colors as colors
import imageio


sys.path.insert(0,'/media/DATA/jbonato/astro_segm/Astro3S/modules/')
from gen_single_astro  import filt_im
from sel_active_reg_gen import *
from model.dense_up import dense_up
from test_fun import gen_sc_mask,fix_mask,prob_calc,small_soma_to_proc,common_merge,art_rem_large,art_rem
from get_traces import Extr_miniROI,get_signal,get_signals
from gui_results import layout


# model to import
model = dense_up(3)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device1 = torch.device('cpu')#('cuda:1' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


###
MAX_ROI_AREA_PROC=30
MU_PX = 1

In [2]:
#PARAMS
N=256
M=256

fov_list = [2]
fov_DNN_weights_folder = '/media/DATA/jbonato/astro_segm/weights/dense_up' 
set_dir='/media/DATA/jbonato/astro_segm/set1/'

In [3]:
class SimDataset_test(Dataset):
    def __init__(self,image_set):
        self.input_images = image_set    


    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):        
        image = self.input_images[idx]
        image = torch.from_numpy(image).float()

        return image

    
vis_flag = False
##### LOAD PARAM DICT
with open(set_dir+'.tmp/dict_dataset1.txt', "rb") as fp:   #Pickling
    dict_param = pickle.load(fp)
max_min = dict_param['max_min']
dict_param


{'list': [0, 40, 80, 120, 160],
 'blocks': 15,
 'threads': 32,
 'BPM_ratio': 3,
 'bb': 96,
 'N_pix_st': 100,
 'astr_min': 80,
 'percentile': 80,
 'pad': 5,
 'astro_num': 4,
 'init_th_': 0.6,
 'decr_dim': 10,
 'decr_th': 25,
 'corr_int': False,
 'gpu_flag': True,
 'max_min': array([345,  89]),
 'th1_p': 0.25,
 'th2_p': 0.1}

In [12]:
dict_im = {}

for jj in fov_list:
    #im_list = []
    Res_1 = np.zeros((N,M,3))

    test_folder_str =str(jj)
    if len(test_folder_str)==1:
        test_folder_str1='00'+test_folder_str
    else:
        test_folder_str1='0'+test_folder_str

    model.load_state_dict(torch.load(fov_DNN_weights_folder+test_folder_str1+'D1.pt'))
    
    #collect stack to analyze
    
    stack_dir = '/media/DATA/jbonato/segm_project/set/'+test_folder_str+'/'

    items_stack = os.listdir(stack_dir)

    print(stack_dir + items_stack[0])
    stack = io.imread(stack_dir + items_stack[0]).astype(np.uint16)
    dict_im['t-series_'+test_folder_str1] = stack
    frames,_,_ = stack.shape

    a_reg = sel_active_reg(stack.astype(np.float32),dict_param)
    mask = a_reg.get_mask()
    mask = fix_mask(mask)
    
    #im_list.append([mask,'Active Regions'])
    dict_im['Active Regions_'+test_folder_str1] = mask
    filter_ = filt_im(stack_dir + items_stack[0],mask,dict_param['bb']-2*dict_param['pad'])
    _, image_to_plot = filter_.create_img()
    # for other dataset spatia_pp methods can be called from filter_, outputs are stack filtered and spatial map enhanced  
    #im_list.append([image_to_plot,'Enhanced'])
    dict_im['Enhanced_'+test_folder_str1] = image_to_plot
    coord_l = filter_.get_instances()
    
    assert coord_l!=0, 'Check Active region extraction module'
    
    
    image_stack = np.empty((len(coord_l),dict_param['bb'],dict_param['bb'])) 

    image_stack,filt_imageL = filter_.save_im()#select the padding val 5 is default
    
    
    image_set = image_stack[:,0,:,:]
    image_set = image_set[:,np.newaxis,:,:]

    imageL_set = image_to_plot*filt_imageL
    imageL_set-=np.mean(imageL_set)
    imageL_set= imageL_set[np.newaxis,np.newaxis,:,:]



    model.eval()   # Set model to evaluate mode
    ######################################## Prob Map
    test_datasetL = SimDataset_test(imageL_set)
    test_loader = DataLoader(test_datasetL, batch_size=15, shuffle=False, num_workers=0)

    inputs = next(iter(test_loader))
    inputs = inputs.to(device)
    pred = model(inputs)
    
    pred_mean = pred.data.cpu().numpy()
    del test_datasetL,test_loader, inputs,pred
    
    mean = pred_mean[0]
    maxim = np.amax(mean,axis=0)
    mean[mean<maxim]=0
        
    prob_mapPL,sm_ent = prob_calc(mean[1,:,:],max_min[0],max_min[1])
    
    #im_list.append([prob_mapPL,'Prob. Map'])
    dict_im['Prob. Map PL_'+test_folder_str1] = prob_mapPL
    #im_list.append([sm_ent,'Prob. Map'])
    dict_im['Prob. Map_'+test_folder_str1] = sm_ent
    ########################################## putative single cell
    test_dataset_S = SimDataset_test(image_set)
    test_loader = DataLoader(test_dataset_S, batch_size=15, shuffle=False, num_workers=0)

    
    pred_mean=[]
    for inputs in test_loader:
        inputs = inputs.to(device)

        pred = model(inputs)
        pred_mean.append(pred.data.cpu().numpy())
        del inputs,pred

    torch.cuda.empty_cache()
    
    for j in range(1,len(pred_mean)):
        pred_mean[0]=np.vstack((pred_mean[0],pred_mean[j]))

    prob_map = np.zeros((N,M,2))

    for i in range(len(coord_l)):
        mean= np.zeros((3,dict_param['bb'],dict_param['bb']))
        mean = pred_mean[0][i,:,:,:].copy()
        
        maxim = np.amax(mean,axis=0)
        mean[mean<maxim]=0
        mean[mean>=maxim]=1

        small_soma = small_soma_to_proc(mean[1,:,:],N = int((2/3)*max_min[1])) ####remove too small somata segmentated
        mean[0,:,:]+=small_soma
        mean[1,:,:]-=small_soma
        
        coord = coord_l[i]
        Res_1[coord[1]:coord[3],coord[0]:coord[2],0] += mean[0,dict_param['pad']:-dict_param['pad'],dict_param['pad']:-dict_param['pad']]
        Res_1[coord[1]:coord[3],coord[0]:coord[2],1] += mean[1,dict_param['pad']:-dict_param['pad'],dict_param['pad']:-dict_param['pad']]



    Res_1[:,:,0] -= Res_1[:,:,1]
    Res_1[Res_1<1]=0
    Res_1[Res_1>0]=1

    ######### can be wrapped
    
    soma_f = common_merge(Res_1[:,:,1],sm_ent)
    Res_1[:,:,1]=soma_f

    #remove possible artifacts
    small_soma = small_soma_to_proc(Res_1[:,:,1],int(0.9*max_min[1]),dilation=False)
    Res_1[:,:,1]-=small_soma

    Res_1[:,:,0] = Res_1[:,:,0]-Res_1[:,:,1]
    Res_1[Res_1<1]=0
    Res_1[Res_1>0]=1

    #remove large region classified as soma Area>500
    Res_1_filt,removal = art_rem_large(Res_1[:,:,1],Res_1[:,:,0],N=int(1.15*max_min[0]))
    if removal<2:
        Res_1-=Res_1_filt[:,:,np.newaxis]

    Res_1_filt,removal = art_rem_large(Res_1[:,:,1],Res_1[:,:,0],N=max_min[0])
    if removal<2:
        Res_1-=Res_1_filt[:,:,np.newaxis]


    #remove processes without soma
    Res_1_filt = art_rem(Res_1[:,:,1],Res_1[:,:,0])
    Res_1*=Res_1_filt[:,:,np.newaxis]
    
    #im_list.append([Res_1,'Final Mask'])
    dict_im['Final_Mask_'+test_folder_str1] = Res_1
    print(20*'%')
    #######################################################################################################
    #Visualization of images
    if vis_flag:
        vis = Visdom(port=8097, server="http://localhost",env='inference_plot')
        for key in dict_im.keys():
            image = dict_im[key]
            fig, ax = plt.subplots(figsize=(4,4))
            ax.imshow(image)
            ax.axis('off')
            ax.set_title(key)
            vis.matplot(fig)
            plt.close(fig)
    
    #### Extraction can be wrapped into a function
    single_astro_roi = gen_sc_mask(dict_im['Final_Mask_'+test_folder_str1])
    dict_im['Single_cell_mask_'+test_folder_str1] = single_astro_roi
    dict_roi={}
    dict_traces={}
    dict_im['Cell_num_'+test_folder_str1] = single_astro_roi.shape[0]
    print("ROI NUM",single_astro_roi.shape[0])
    for s_roi_num in range(single_astro_roi.shape[0]):
        print(50*'%','Extracting cell:',s_roi_num)
        constr_split_roi = Extr_miniROI(MAX_ROI_AREA_PROC,MU_PX,single_astro_roi[s_roi_num,:,:,0],single_astro_roi[s_roi_num,:,:,1],True)
        arr_out_proc = constr_split_roi.get_miniROI()
        if  s_roi_num==0:
            list_out=arr_out_proc
        else:
            list_out = np.dstack((list_out,arr_out_proc))
        name = str(s_roi_num)
        dict_roi['Soma_'+f'{name:0>3}'] = np.where(single_astro_roi[s_roi_num,:,:,1]==1)
        dict_traces['Soma_'+f'{name:0>3}'] = get_signal(single_astro_roi[s_roi_num,:,:,1],dict_im['t-series_'+test_folder_str1])
        for proc_num in range(arr_out_proc.shape[2]):
            name_proc = str(proc_num)
            dict_roi['Proc_'+f'{name:0>3}'+'_'+f'{name_proc:0>3}'] = np.where(arr_out_proc[:,:,proc_num]==1)
            dict_traces['Proc_'+f'{name:0>3}'+'_'+f'{name_proc:0>3}']  = get_signal(arr_out_proc[:,:,proc_num],dict_im['t-series_'+test_folder_str1])
        print('Extraction: done')
    
    dict_im['Signals_extr_'+test_folder_str1] = dict_traces
    dict_im['ROI_'+test_folder_str1] = dict_roi
    
    #### for Visualization purposes
    list_out = np.dstack((list_out,dict_im['Final_Mask_'+test_folder_str1][:,:,1:]))
    dict_im['Final_Mask_fraction_'+test_folder_str1] = list_out
    #### save dict
    pickle.dump(dict_im, open( "inference_ex.pkl", "wb" ) )

/media/DATA/jbonato/segm_project/set/2/TSeries-04082019-1513-1251_Ch2__movie_corrected_aligned.tiff
Zones 10
134 0.91
qw 2
qw 43
157 0.93
154 0.9
135 0.93
qw 7
161 0.94
qw 27
qw 1
qw 35
qw 1
qw 2
qw 5
%%%%%%%%%%%%%%%%%%%%
ROI NUM 5
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Extracting cell: 0
ROIS 4
mat_out (4, 256, 256)
check size (4, 256, 256)
ratio 7.3 7
ratio 0.43333333333333335 0
ratio 1.4 1
ratio 0.7666666666666667 0
SPLIT DONE (256, 256, 17)
Extraction: done
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Extracting cell: 1
ROIS 1
mat_out (1, 256, 256)
check size (1, 256, 256)
ratio 24.933333333333334 24
SPLIT DONE (256, 256, 48)
Extraction: done
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Extracting cell: 2
ROIS 4
mat_out (4, 256, 256)
check size (4, 256, 256)
ratio 3.033333333333333 3
ratio 2.9 2
ratio 3.3333333333333335 3
ratio 2.8 2
SPLIT DONE (256, 256, 13)
Extraction: done
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Extracting cell: 3
ROIS 4
mat_ou

In [5]:
list_out.shape

(256, 256, 111)

In [5]:
dict_im.keys()

dict_keys(['t-series_002', 'Active Regions_002', 'Enhanced_002', 'Prob. Map PL_002', 'Prob. Map_002', 'Final_Mask_002', 'Cell_num_002', 'Signals_extr_002', 'ROI_002', 'Final_Mask_fraction_002'])

In [7]:
dict_im['ROI_002'].keys()

dict_keys(['Soma_000', 'Proc_000_000', 'Proc_000_001', 'Proc_000_002', 'Proc_000_003', 'Proc_000_004', 'Proc_000_005', 'Proc_000_006', 'Proc_000_007', 'Proc_000_008', 'Proc_000_009', 'Proc_000_010', 'Proc_000_011', 'Proc_000_012', 'Proc_000_013', 'Proc_000_014', 'Proc_000_015', 'Proc_000_016', 'Soma_001', 'Proc_001_000', 'Proc_001_001', 'Proc_001_002', 'Proc_001_003', 'Proc_001_004', 'Proc_001_005', 'Proc_001_006', 'Proc_001_007', 'Proc_001_008', 'Proc_001_009', 'Proc_001_010', 'Proc_001_011', 'Proc_001_012', 'Proc_001_013', 'Proc_001_014', 'Proc_001_015', 'Proc_001_016', 'Proc_001_017', 'Proc_001_018', 'Proc_001_019', 'Proc_001_020', 'Proc_001_021', 'Proc_001_022', 'Proc_001_023', 'Proc_001_024', 'Proc_001_025', 'Proc_001_026', 'Proc_001_027', 'Proc_001_028', 'Proc_001_029', 'Proc_001_030', 'Proc_001_031', 'Proc_001_032', 'Proc_001_033', 'Proc_001_034', 'Proc_001_035', 'Proc_001_036', 'Proc_001_037', 'Proc_001_038', 'Proc_001_039', 'Proc_001_040', 'Proc_001_041', 'Proc_001_042', 'Proc

In [10]:
dict_im['Final_Mask_fraction_002'].shape

(256, 256, 111)

In [8]:
fov_name = []
for j in fov_list:
    j = str(j)
    if len(j)==1:
        j='00'+j
    else:
        j='0'+j
    fov_name.append(j)
fov_name
hbox,button,display_plot = layout(fov_name,dict_im)
display(hbox)
button.on_click(display_plot)

['002']

In [14]:
dict_im['ROI_002'].keys()

dict_keys(['Soma_000', 'Proc_000_000', 'Proc_000_001', 'Proc_000_002', 'Proc_000_003', 'Proc_000_004', 'Proc_000_005', 'Proc_000_006', 'Proc_000_007', 'Proc_000_008', 'Proc_000_009', 'Proc_000_010', 'Proc_000_011', 'Proc_000_012', 'Proc_000_013', 'Proc_000_014', 'Proc_000_015', 'Proc_000_016', 'Soma_001', 'Proc_001_000', 'Proc_001_001', 'Proc_001_002', 'Proc_001_003', 'Proc_001_004', 'Proc_001_005', 'Proc_001_006', 'Proc_001_007', 'Proc_001_008', 'Proc_001_009', 'Proc_001_010', 'Proc_001_011', 'Proc_001_012', 'Proc_001_013', 'Proc_001_014', 'Proc_001_015', 'Proc_001_016', 'Proc_001_017', 'Proc_001_018', 'Proc_001_019', 'Proc_001_020', 'Proc_001_021', 'Proc_001_022', 'Proc_001_023', 'Proc_001_024', 'Proc_001_025', 'Proc_001_026', 'Proc_001_027', 'Proc_001_028', 'Proc_001_029', 'Proc_001_030', 'Proc_001_031', 'Proc_001_032', 'Proc_001_033', 'Proc_001_034', 'Proc_001_035', 'Proc_001_036', 'Proc_001_037', 'Proc_001_038', 'Proc_001_039', 'Proc_001_040', 'Proc_001_041', 'Proc_001_042', 'Proc

In [15]:
dict_im['ROI_002']['Soma_000']

(array([11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20]),
 array([79, 80, 81, 78, 79, 80, 81, 82, 83, 84, 88, 89, 90, 91, 77, 78, 79,
        80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 76, 77, 78,
        79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 76,
        77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93,
        76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92,
        93, 77